**COUNTERFACTUALS GUIDED BY PROTOTYPES ON BREAST CANCER DATASET**

Counterfactual explanations : Smallest change that can be made to the feature(/s) so as to get our desired outcome.
- For example, if a person is applying for jobs in different companies along with his colleagues, and if he is facing some rejections while the others are getting jobs, then Counterfactual explanations give the idea of what small change(/s) (i.e. skills) he should develop in order to be in the same league with his colleagues.
- Note: There can be more than one counterfactuals, as we can have different changes on different features which result to our desired outcome

Counterfactuals Guided by Prototypes: The counterfactuals guided by prototypes method works on black-box models.

In [ ]:
!pip install alibi

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from alibi.explainers import CounterFactualProto

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.3.0
Eager execution enabled:  False


Loading the dataset:

In [ ]:
dataset = load_breast_cancer()
data = dataset.data
target = dataset.target
feature_names = dataset.feature_names
print(feature_names)
print(len(feature_names))

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
30


Standardizing the data:

In [ ]:
print(data[:][0])
mu = data.mean(axis=0)
sigma = data.std(axis=0)
data = (data - mu) / sigma

[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]


In [ ]:
print(data[:][0])

[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


Defining train and test set:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, target,random_state=0)
#print(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
#print(y_test)
#print(len(x_test))

Training the Model:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def nn_model():
    x_in = Input(shape=(30,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [ ]:
nn = nn_model()
nn.summary()
nn.fit(x_train, y_train, batch_size=64, epochs=500, verbose=0)
nn.save('nn_breast_cancer.h5', save_format='h5')

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 40)                1240      
_________________________________________________________________
dense_4 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 82        
Total params: 2,962
Trainable params: 2,962
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nn = load_model('nn_breast_cancer.h5')
score = nn.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', score[1])
print(score)

Test accuracy:  0.97202796
[0.09764901470471095, 0.97202796]


The accuracy is 96%, which indicates that our model is quite good.

Original instance:

In [ ]:
#print(x_test[1],y_test[1])
#print(x_test[1].shape)
#print("L",(1,), x_test[1].shape)
X = x_test[2].reshape((1,) + x_test[2].shape)
print(X)
shape = X.shape
print(shape)
print(x_train.shape)

[[-0.03047238 -0.84464357 -0.09799286 -0.13762358 -1.18848289 -0.91973395
  -0.8528506  -0.57776317 -0.81276801 -0.98347839 -0.68925826 -1.01957677
  -0.62376789 -0.46395179 -0.73468942 -0.90765388 -0.75202503 -0.16807145
  -1.06940152 -0.63958666 -0.28146446 -1.03686277 -0.31963817 -0.33696221
  -1.26986414 -0.97052684 -1.00550568 -0.49404558 -1.23720696 -0.93352473]]
(1, 30)
(426, 30)


Run Counterfactual:

In [ ]:
#define model
nn = load_model('nn_breast_cancer.h5')

# initialize explainer, fit and generate counterfactual
cf = CounterFactualProto(nn, shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=(x_train.min(axis=0), x_train.max(axis=0)),
                         c_init=1., c_steps=10)

cf.fit(x_train)
explanation = cf.explain(X,k=1)
print((explanation))

No encoder specified. Using k-d trees to represent class prototypes.


Explanation(meta={
    'name': 'CounterFactualProto',
    'type': ['blackbox', 'tensorflow', 'keras'],
    'explanations': ['local'],
    'params': {
        'write_dir': None,
        'update_num_grad': 1,
        'clip': (-1000.0, 1000.0),
        'eps': (0.001, 0.001),
        'c_steps': 10,
        'c_init': 1.0,
        'max_iterations': 1000,
        'learning_rate_init': 0.01,
        'use_kdtree': True,
        'ohe': False,
        'cat_vars': None,
        'theta': 10.0,
        'gamma': 0.0,
        'feature_range': (
            array([-2.0296483 , -2.22924851, -1.98450403, -1.45444309, -3.11208479,
       -1.61013634, -1.11487284, -1.26181958, -2.74411707, -1.81986478,
       -1.05992413, -1.54954662, -1.04404888, -0.72845634, -1.77606498,
       -1.2980982 , -1.05750068, -1.91344745, -1.53289003, -1.09696818,
       -1.72690052, -2.22399401, -1.69336103, -1.22242284, -2.68269492,
       -1.4438784 , -1.30583065, -1.74506282, -2.16095969, -1.60183949]),
            array([

In [ ]:
print('Original prediction: {}'.format(explanation.orig_class))
print('Counterfactual prediction: {}'.format(explanation.cf['class']))

Original prediction: 1
Counterfactual prediction: 0


The original prediction is 1 that is benign cancer. On applying counterfactual, the prediction class is changed to 0 by bringing changes in the features.

Now, the differences between the original and counterfactual are shown. Only the features in which a significant change is brought(>0.0001) are printed.

To make the results more interpretable, we will first undo the pre-processing step and then check where the counterfactual differs from the original instance:

In [ ]:
orig = X * sigma + mu
counterfactual = explanation.cf['X'] * sigma + mu
delta = counterfactual - orig
ans=[]
for i, f in enumerate(feature_names):
    #if np.abs(delta[0][i]) > 1e-4:
        ans.append([f, delta[0][i]])
        #print('{}: {}'.format(f, delta[0][i]))
ans.sort(reverse=True,key = lambda x: x[1])
for i in ans:
  print(i)

['worst area', 158.44928861929702]
['area error', 8.199435012531097]
['worst texture', 7.857185849354991]
['worst perimeter', 4.658540297260757]
['mean texture', 1.9676821473195112]
['worst radius', 1.3743336786874814]
['perimeter error', 0.6160816403278029]
['radius error', 0.2103921495599966]
['texture error', 0.1822691665420897]
['worst concavity', 0.09506181672364672]
['worst symmetry', 0.059329696138763766]
['worst smoothness', 0.026328729806061033]
['worst concave points', 0.014116083099844315]
['mean concavity', 0.010440591437178792]
['mean smoothness', 0.001462515602263223]
['mean perimeter', 4.8720124823375954e-08]
['worst compactness', 3.7052776125090503e-09]
['concavity error', 7.21307798212667e-10]
['mean compactness', 3.1492434354740695e-10]
['compactness error', 2.127324878753445e-10]
['mean fractal dimension', 1.3257168279823262e-10]
['symmetry error', 1.2747810446134267e-10]
['worst fractal dimension', 8.046209531986648e-11]
['fractal dimension error', 7.759411429358876

In [ ]:
orig = X * sigma + mu
counterfactual = explanation.cf['X'] * sigma + mu
delta = counterfactual - orig
for i, f in enumerate(feature_names):
    #if np.abs(delta[0][i]) > 1e-4:
    print('{}: {}'.format(f, delta[0][i]))

mean radius: -1.9393553429836174e-10
mean texture: 1.9676821473195112
mean perimeter: 4.8720124823375954e-08
mean area: -7.143080438254401e-07
mean smoothness: 0.001462515602263223
mean compactness: 3.1492434354740695e-10
mean concavity: 0.010440591437178792
mean concave points: -1.0494254285009497e-09
mean symmetry: -7.560765624692323e-10
mean fractal dimension: 1.3257168279823262e-10
radius error: 0.2103921495599966
texture error: 0.1822691665420897
perimeter error: 0.6160816403278029
area error: 8.199435012531097
smoothness error: 2.925894856259381e-11
compactness error: 2.127324878753445e-10
concavity error: 7.21307798212667e-10
concave points error: 5.273134359717879e-12
symmetry error: 1.2747810446134267e-10
fractal dimension error: 7.759411429358876e-11
worst radius: 1.3743336786874814
worst texture: 7.857185849354991
worst perimeter: 4.658540297260757
worst area: 158.44928861929702
worst smoothness: 0.026328729806061033
worst compactness: 3.7052776125090503e-09
worst concavity:

The above features are changed to change the prediction from 0 to 1.

Loss = cLpred + βL1 + L2 + Lae + Lproto


In [ ]:
counterfactual

array([[1.32100000e+01, 2.52500000e+01, 8.41000003e+01, 5.37900001e+02,
        8.94692029e-02, 5.76879976e-02, 4.03900542e-02, 2.69229741e-02,
        1.61899999e-01, 5.58400001e-02, 3.33116893e-01, 1.35000000e+00,
        2.09136650e+00, 3.33862824e+01, 5.76800003e-03, 8.08199971e-03,
        1.51000002e-02, 6.45100008e-03, 1.34700001e-02, 1.82799996e-03,
        1.58810443e+01, 3.42300000e+01, 9.89184772e+01, 7.11918444e+02,
        1.28900000e-01, 1.17938496e-01, 2.50346116e-01, 7.19726966e-02,
        2.44400001e-01, 6.78800000e-02]])

In [ ]:
orig

array([[1.321e+01, 2.525e+01, 8.410e+01, 5.379e+02, 8.791e-02, 5.205e-02,
        2.772e-02, 2.068e-02, 1.619e-01, 5.584e-02, 2.084e-01, 1.350e+00,
        1.314e+00, 1.758e+01, 5.768e-03, 8.082e-03, 1.510e-02, 6.451e-03,
        1.347e-02, 1.828e-03, 1.435e+01, 3.423e+01, 9.129e+01, 6.329e+02,
        1.289e-01, 1.063e-01, 1.390e-01, 6.005e-02, 2.444e-01, 6.788e-02]])

In [ ]:
x_test[1]

array([-0.26052388,  1.3870138 , -0.32412706, -0.33272902, -0.601368  ,
       -0.99099151, -0.76684905, -0.72840003, -0.70323971, -0.98631359,
       -0.71019168,  0.24157367, -0.76831683, -0.50069465, -0.42434351,
       -0.97226219, -0.55682964, -0.8670329 , -0.85630335, -0.74398403,
       -0.39743068,  1.3927666 , -0.47571596, -0.43540532, -0.15204891,
       -0.94126441, -0.63897488, -0.83070565, -0.73893053, -0.89030039])

In [ ]:
y_test[1]

array([0., 1.], dtype=float32)

In [ ]:
os.remove('nn_breast_cancer.h5')